![Figure](hextruss-irregular.svg)

In [5]:
from sdtruss import sdtruss

jc = [ ('a', 0, 0),
       ('b', 0, 4),
       ('c', 2, 8),
       ('d', 6, 8.001),
       ('e', 8, 4),
       ('f', 8, 0),
       ]

mi = [ ('a', 'b',),
       ('b', 'c',),
       ('c', 'd',), 
       ('d', 'e',),
       ('e', 'f',),
       ('f', 'a',),
       ('b', 'e',),
       ('c', 'f',),
       ('a', 'd',),
       ]

rf = [ ('a', 1, 0),             # reaction force dirns
       ('a', 0, 1),
       ('f', 0, 1),
       ]

jl = [ ('c', 1, 1, 0),      # joint loads (j, p, dx, dy)
       ]

sdtruss( jc, mi, rf, jl )

uid   iid   jid        force
---   ---   ---        -----
ab    a     b           1778 C
bc    b     c           1987 C
cd    c     d           2222 C
de    d     e           1988 C
ef    e     f           1778 C
fa    f     a           1333 C
be    b     e          888.8 T
cf    c     f           2221 T
ad    a     d           2223 T
Rax   a                   -1 
Ray   a                   -1 
Rfy   f                    1 
